# MNIST卷积神经网络

MNIST 包括6万张28x28的训练样本，1万张测试样本，可以说是CV里的“Hello Word”。这里使用的CNN网络将MNIST数据的识别率提高到了99%。下面我们就开始进行实战。

## 测试gpu环境

In [1]:
import torch
torch.cuda.is_available()

True

## 导入包

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFunction
import torch.optim as optim
from torchvision import datasets, transforms

## 定义超参数 

In [3]:
batch_size =512 #设置每一个batch的数量为512
num_epochs =20  # 训练20次
#learning_rate = 1e-4    # 学习率

## 加载数据集

In [4]:
# 加载训练集
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("data", train=True, download=True, 
            transform=transforms.Compose([
              transforms.ToTensor(), # 将输入转换为Tensor的格式
              transforms.Normalize((0.1307,), (0.3081,)) # 用transforms.Normalize对数据进行归一化，参数为数据的平均数和标准差
            ])),
    batch_size=batch_size , shuffle=True) # suffle打乱

# 加载测试集，按相同的方法
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("data", train=False, transform=transforms.Compose([
              transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,))
            ])),
    batch_size=batch_size , shuffle=True)

## 定义卷积神经网络

In [5]:
# 包含两个卷积层，第一层卷积核大小5*5，个数10；第二层卷积核大小3*3，个数20，步长均为1
# 卷积层输出使用relu激活函数处理
# 处理后用2*2的最大值池化，步长为1
# 最后用两个全连接层，第一个输入节点数量20*10*10，输出节点数量500；第二个输入节点数量500，输出节点数量10（即对应10分类）
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_layer1=nn.Conv2d(1,6,5) # input:(1,28,28) output:(6,24,24) 
    self.conv_layer2=nn.Conv2d(10,16,3) # input:(10,12,12) output:(20,10,10)
    self.full_connected_layer1 = nn.Linear(16*10*10,500)
    self.full_connected_layer2 = nn.Linear(500,10)
  def forward(self,x): # 前向计算数值
    in_size = x.size(0)
    out = self.conv_layer1(x)
    out = nnFunction.relu(out)
    out = nnFunction.max_pool2d(out, 2, 2) 
    out = self.conv_layer2(out)
    out = nnFunction.relu(out)
    out = out.view(in_size,-1)
    out = self.full_connected_layer1(out)
    out = nnFunction.relu(out)
    out = self.full_connected_layer2(out)
    out = nnFunction.log_softmax(out,dim=1)
    return out

## 实例化神经网络

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # 根据电脑配置自动选择使用cpu或者gpu
model = ConvNet().to(DEVICE) # 将网络移动到gpu上
optimizer = optim.Adam(model.parameters()) # 使用Adam优化器

## 定义训练函数

In [7]:
# 用到反向传播，Adam优化训练
def train(model, device, train_loader, optimizer, num_epochs):
  model.train()
  for batch_index, (data, target) in enumerate(train_loader):
    # 取数据
    data, target = data.to(device), target.to(device)
    # 先清空所有参数的梯度缓存，否则会在上面累加
    optimizer.zero_grad()
    # 向网络中输入images，得到output,在这一步的时候模型会自动调用model.forward(images)函数 
    output = model(data)
     # 计算损失
    loss = nnFunction.nll_loss(output, target)
    # 反向传播
    loss.backward()
    # 更新梯度
    optimizer.step()
    if(batch_index+1)%30 == 0: 
      print("Train Epoch_{} [{}/{} ({:.0f}%)]	Loss: {:.6f}".format(
        num_epochs, batch_index * len(data), len(train_loader.dataset),
        100. * batch_index / len(train_loader), loss.item()))

## 定义训练集准确率计算函数

In [8]:
def get_train_accuracy(model, device, train_loader):
  #将模型设为评估模式，在模型中禁用dropout或者batch normalization层
  model.eval()
  train_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in train_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      train_loss += nnFunction.nll_loss(output, target, reduction="sum").item() # 将一批的损失相加
      pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
      correct += pred.eq(target.view_as(pred)).sum().item()

  train_loss /= len(train_loader.dataset)
  print("train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(train_loss, correct, len(train_loader.dataset),100. * correct / len(train_loader.dataset)))

## 定义测试集测试函数

In [9]:
def test(model, device, test_loader):
  #将模型设为评估模式，在模型中禁用dropout或者batch normalization层
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += nnFunction.nll_loss(output, target, reduction="sum").item() # 将一批的损失相加
      pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print("Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))

## 开始训练 

In [10]:
for epoch in range(1, num_epochs  + 1):
  train(model, DEVICE, train_loader, optimizer, epoch)
  get_train_accuracy(model, DEVICE, train_loader)
  test(model, DEVICE, test_loader)
  print("")

Train Epoch_1 [14848/60000 (25%)]	Loss: 0.290438
Train Epoch_1 [30208/60000 (50%)]	Loss: 0.193169
Train Epoch_1 [45568/60000 (75%)]	Loss: 0.142141
train set: Average loss: 0.0974, Accuracy: 58297/60000 (97.16%)
Test set: Average loss: 0.0898, Accuracy: 9721/10000 (97.21%)

Train Epoch_2 [14848/60000 (25%)]	Loss: 0.067529
Train Epoch_2 [30208/60000 (50%)]	Loss: 0.072156
Train Epoch_2 [45568/60000 (75%)]	Loss: 0.089861
train set: Average loss: 0.0661, Accuracy: 58754/60000 (97.92%)
Test set: Average loss: 0.0598, Accuracy: 9804/10000 (98.04%)

Train Epoch_3 [14848/60000 (25%)]	Loss: 0.085427
Train Epoch_3 [30208/60000 (50%)]	Loss: 0.064826
Train Epoch_3 [45568/60000 (75%)]	Loss: 0.057574
train set: Average loss: 0.0406, Accuracy: 59302/60000 (98.84%)
Test set: Average loss: 0.0478, Accuracy: 9844/10000 (98.44%)

Train Epoch_4 [14848/60000 (25%)]	Loss: 0.038545
Train Epoch_4 [30208/60000 (50%)]	Loss: 0.043670
Train Epoch_4 [45568/60000 (75%)]	Loss: 0.031260
train set: Average loss: 0.0340